# Libraries

In [1]:
import pandas as pd
from datetime import datetime
import pyodbc
import shutil
import os
from openpyxl import load_workbook

# Parameters

In [2]:
# Ruta de la data de Cruz Verde
path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\Colombia\Cruz Verde\2021'

# Año y Mes a transformar
anio = 2021
mes = 'Abril'

# Nombre de los archivos de Cruz Verde
filename_so = r'\Rotacion_CruzVerde_900199922_Consumo_Farma_A_202104.xlsx'
filename_st = r'\Inventario_CruzVerde_900199922_Consumo_Farma_A_202104.xlsx'

# Ruta del layout de ISV No B2B
path_original_layout = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\01Code\01ISV\COL\1Data\2Catalogue\toISV'

## Data Base

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

Nos traemos el primer domingo del mes Genomma:

In [4]:
mes_numero = {
    'Enero':1,
    'Febrero':2,
    'Marzo':3,
    'Abril':4,
    'Mayo':5,
    'Junio':6,
    'Julio':7,
    'Agosto':8,
    'Septiembre':9,
    'Octubre':10,
    'Noviembre':11,
    'Diciembre':12
}

In [5]:
qfecha = '''
SELECT MIN(TmpFecha) Fecha
FROM Gnm_DWH.dbo.Dim_Tiempo
WHERE TmpAnioSemanaGenomma = {0} AND TmpMesGenomma = {1} AND TmpNombreDiaSemana = 'Domingo';
'''

In [6]:
fecha = pd.read_sql(qfecha.format(anio, mes_numero[mes]), conn1)

# Load Data

**df_so**: Hace referecia a los datos de "Sell Out" <br>
**df_st**: Hace referecia a los datos de "Stock"

In [7]:
%%time
df_so = pd.read_excel(path + '\\' + mes + filename_so)
df_st = pd.read_excel(path + '\\' + mes + filename_st)

Wall time: 12.5 s


# Clean Data

## Sell Out

In [8]:
data_so = df_so[['Código Producto', 'Código de Barras', 'Descripción Producto', 'Codigo Org Inventario', 'Nombre Org Inventario', 'Unidades vendidas']].copy()

# Agregamos dos columnas nuevas
data_so['Cadena'] = 'Cruz Verde'
data_so['Fecha_Venta'] = fecha.loc[0, 'Fecha']

In [9]:
names_so = {'Código Producto':'Cod_Prod de la Cadena',
            'Código de Barras':'EAN o UPC Genomma',
            'Descripción Producto':'Descripcion_Prod',
            'Codigo Org Inventario':'Cod_Local',
            'Nombre Org Inventario':'Descripcion_Local',
            'Unidades vendidas':'Unidades'}

data_so.rename(names_so, axis=1, inplace=True)

In [10]:
data_so = data_so.pivot_table(index=['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena',
                                     'EAN o UPC Genomma', 'Descripcion_Prod',
                                     'Cod_Local', 'Descripcion_Local'],
                              values=['Unidades'], aggfunc='sum').reset_index()

Ya tenemos lista la información de Sell Out en el layout de ISV.

## Stocks

In [11]:
data_st = df_st[['ARTICULO', 'EAN', 'DESCRIPCION', 'CODIGO ORGANIZACION', 'ORGANIZACION', 'CANTIDAD ARTICULO']].copy()

# Agregamos dos columnas nuevas
data_st['Cadena'] = 'Cruz Verde'
data_st['Fecha_Stock'] = fecha.loc[0, 'Fecha']

In [12]:
names_st = {'ARTICULO':'Cod_Prod', 'EAN':'EAN o UPC Genomma',
            'DESCRIPCION':'Descripcion_Prod',
            'CODIGO ORGANIZACION':'Cod_Local',
            'ORGANIZACION':'Descripcion_Local',
            'CANTIDAD ARTICULO':'Unidades'}

data_st.rename(names_st, axis=1, inplace=True)

In [13]:
data_st = data_st.pivot_table(index=['Fecha_Stock', 'Cadena', 'Cod_Prod',
                                     'EAN o UPC Genomma', 'Descripcion_Prod',
                                     'Cod_Local', 'Descripcion_Local'],
                              values=['Unidades'], aggfunc='sum').reset_index()

Ya tenemos lista la información de Inventarios en el layout de ISV.

## General

Generamos algunos últimos cambios en las columnas de Sell Out e Inventario:

In [14]:
# Cambio a str de EAN y Cod_Local
data_so['EAN o UPC Genomma'] = data_so['EAN o UPC Genomma'].map(int).map(str)
data_so['Cod_Local'] = data_so['Cod_Local'].map(int).map(str)

data_st['EAN o UPC Genomma'] = data_st['EAN o UPC Genomma'].map(int).map(str)
data_st['Cod_Local'] = data_st['Cod_Local'].map(int).map(str)

# Export to ISV

In [15]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

Creando un nuevo folder donde se guardará el archivo ISV No B2B con la información de CV.

In [16]:
new_folder = path_original_layout + '\\{0}\\{1}'.format(anio, mes)

if not os.path.exists(new_folder):
    os.mkdir(new_folder)

Creando la ruta de donde se encuentra el layout de ISV y donde vamos a exportar la información

In [17]:
path_original_layout = path_original_layout + '\\Formato No B2B Orig.xlsx'
filename = f'Formato No B2B Cruz Verde {mes} - {anio}.xlsx'
path_export = new_folder + '\\' + filename

Exportación tanto de sell out como inventario:

In [18]:
%%time
shutil.copy(path_original_layout, path_export)
write_layout(path_export, data_so, True)
write_layout(path_export, data_st, False)

Wall time: 30.8 s
